In [1]:
from difflib import get_close_matches
import csv
from typing import Any, Optional, Set, Dict, List, Union
from dataclasses import dataclass, field
from pgmpy.readwrite import BIFReader
import numpy as np
from collections import defaultdict, deque, Counter
import os

# Parameters
GROUP_ID = 29
ALGORITHM = 'gibbs'
NETWORK_NAME = './Networks/insurance.bif'
REPORT = '[MedCost, ILiCost, PropCost]'
EVIDENCE_LEVEL = 'None'
EVIDENCE = \
"""
"""

EVIDENCE = EVIDENCE.replace("\n","")
REPORT_DELIM = ","
for char in "“”\"":
    EVIDENCE = EVIDENCE.replace(char, "")

@dataclass (frozen=True)
class Node:
    # name is primary key
    name: str
    parents: tuple
    values: tuple
    probability_model: Optional[np.ndarray] = field(default=None, compare=False, repr=False)

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        if isinstance(other, Node):
            return self.name == other.name
        return False

    def __str__(self):
        return f'{self.name}: [{", ".join(self.values)}]'

class Network:
    def __init__(self, nodes: Set[Node] | None = None):
        self.nodes: Set[Node] = nodes or set()
        self.parents: Dict[Node, Set[Node]] = defaultdict(set)   # child -> {parents}
        self.children: Dict[Node, Set[Node]] = defaultdict(set)  # parent -> {children}
        self.by_name: Dict[str, Node] = {}

    def add_node(self, node: Node):
        if node.name not in self.by_name:
            self.by_name[node.name] = node
            self.nodes.add(node)

    def add_edge(self, parent_name: str, child_name: str):
        parent = self.by_name[parent_name]
        child = self.by_name[child_name]
        self.parents[child].add(parent)
        self.children[parent].add(child)

    def markov_blanket(self, node: Node) -> Set[Node]:
        blanket: Set[Node] = self.parents.get(node, set())
        children = self.children.get(node, set())
        blanket.update(children)
        # children's parents excluding self
        for child in children:
            blanket.update(parent for parent in self.parents.get(child, set()) if parent is not node)
        blanket.discard(node)
        return blanket

    def degree(self, node: Node) -> int:
        return len(self.parents[node])+len(self.children[node])

    def topological_order(self):
        degree_in_context = {node:len(self.parents[node]) for node in self.nodes}
        node_queue = deque(sorted(self.nodes, key=lambda node: degree_in_context[node]))

        output = []
        while node_queue:
            current_node = node_queue.pop()
            if degree_in_context[current_node] == 0:
                output.append(current_node)
                for child in self.children[current_node]:
                    degree_in_context[child] -= 1
            else:
                node_queue.appendleft(current_node)

        return output



    def __str__(self):
        return "\n".join(str(n) for n in sorted(self.nodes, key=lambda n: n.name))

class InputReader:
    def __init__(self):
        reader = BIFReader(NETWORK_NAME)
        model = reader.get_model()
        states = reader.get_states()
        net = Network()

        # 1) add nodes with CPDs reshaped to (child, *parents)
        for variable in model.nodes():
            cpd = model.get_cpds(variable)

            # Get parents in the SAME order as pgmpy stores them
            parents_from_cpd = cpd.variables[1:]  # First is child, rest are parents
            parents = tuple(str(p) for p in parents_from_cpd)  # Use pgmpy's order!

            child_card = len(states[variable])
            parent_cards = [len(states[p]) for p in parents]

            pm_nd = np.array(cpd.values, dtype=float).reshape(
                (child_card, *parent_cards),
                order="F"
            )

            net.add_node(Node(
                name=str(variable),
                parents=parents,
                values=tuple(states[variable]),
                probability_model=pm_nd
            ))

        # 2) add edges
        for child in model.nodes():
            cpd = model.get_cpds(child)
            for parent in (cpd.get_evidence() or []):
                net.add_edge(str(parent), str(child))

        self.network = net

        # 3) parse EVIDENCE string
        self.parsed_evidence = {}
        if EVIDENCE:
            for statement in EVIDENCE.split(';'):
                i = statement.find('=')
                key = statement[:i].strip()
                value = statement[i+1:].strip()
                if key not in self.network.by_name:
                    raise Exception("Invalid Evidence " + key)
                self.parsed_evidence[key] = value

        # 4) parse report
        self.parsed_report = []
        self.parsed_report = REPORT
        self.parsed_report = self.parsed_report[1:] if self.parsed_report.startswith("[") else self.parsed_report
        self.parsed_report = self.parsed_report[:-1] if self.parsed_report.endswith("]") else self.parsed_report
        self.parsed_report = [s.strip() for s in self.parsed_report.split(REPORT_DELIM)]

class Factor:
    def __init__(self, variables, values):

        self.variables = list(variables)
        self.values = np.array(values, dtype=np.float32)
        assert self.values.ndim == len(self.variables), \
            f"values.ndim ({self.values.ndim}) must equal len(variables) ({len(self.variables)})."

    def reorder(self, vars_order):
        """Reorder axes to match vars_order."""
        if set(vars_order) != set(self.variables):
            raise ValueError(
                f"reorder mismatch. have={self.variables}, want={vars_order}"
            )
        source = list(range(len(self.variables)))
        destination = [vars_order.index(v) for v in self.variables]
        self.values = np.moveaxis(self.values, source, destination)
        self.variables = list(vars_order)
        return self

    def restrict(self, var, value_index):
        if var not in self.variables:
            return self
        ax = self.variables.index(var)
        self.values = np.take(self.values, indices=value_index, axis=ax)
        self.variables.pop(ax)
        return self

    def _align_to(self, all_vars):
        """Return a view to order all_vars."""
        dest_axes = [all_vars.index(v) for v in self.variables]

        arr = self.values
        need = len(all_vars) - arr.ndim
        if need > 0:
            arr = arr.reshape(arr.shape + (1,) * need)

        arr = np.moveaxis(arr, list(range(len(self.variables))), dest_axes)
        return arr

    def multiply(self, other):
        """Pointwise multiply after aligning axes by variable name."""
        all_vars = list(dict.fromkeys(self.variables + other.variables))
        A = self._align_to(all_vars)
        B = other._align_to(all_vars)
        prod = A * B  # numpy handles missing vars
        return Factor(all_vars, prod)

    def sum_out(self, var):
        if var not in self.variables:
            return self
        ax = self.variables.index(var)
        self.values = self.values.sum(axis=ax)
        self.variables.pop(ax)
        return self

    def normalize(self):
        Z = self.values.sum()
        if Z != 0:
            self.values = self.values / Z
        # else: leave as-is
        return self

class VESolver:
    @staticmethod
    def _value_index(node, label: str) -> int:
        try:
            return node.values.index(label)
        except ValueError:
            lower_vals = [s.lower() for s in node.values]
            try:
                return lower_vals.index(label.lower())
            except ValueError:
                allowed = ", ".join(node.values)
                raise ValueError(
                    f"Value {label!r} invalid for {node.name}. Allowed: [{allowed}]"
                )

    def _factor_from_node(self, network, node) -> "Factor":
        """
        Assumes CPD stored as shape (child, *parents)
        """
        child = node.name
        parents = list(node.parents or ())
        variables = [child] + parents
        vals = np.array(node.probability_model, dtype=float, copy=True)
        return Factor(variables, vals)

    def _build_factors(self, network) -> list["Factor"]:
        return [self._factor_from_node(network, n) for n in network.nodes]

    def _ancestors_of(self, network, vars_set: set[str]) -> set[str]:
        """
        Return vars_set ∪ all their (recursive) parents
        """
        keep = set(vars_set)
        stack = list(vars_set)
        while stack:
            v = stack.pop()
            node = network.by_name[v]
            for p in (node.parents or ()):
                if p not in keep:
                    keep.add(p)
                    stack.append(p)
        return keep

    def _build_factors_for(self, network, keep_vars: set[str]) -> list["Factor"]:
        """Only build factors for nodes whose variable is in keep_vars."""

        return [
            self._factor_from_node(network, n)
            for n in network.nodes
            if n.name in keep_vars
        ]

    def solve(self, network, query: List[str] | str, evidence: dict[str, str]):
        if isinstance(query, str):
            query = [query]

        if query[0] in evidence:
            node = network.by_name[query[0]]
            num_values = len(node.values)
            return Factor([query[0]], [-1]*num_values)

        # Map evidence labels, indices (case-insensitive, validated)
        evidence_node_states = {}
        for variable, label in evidence.items():
            node = network.by_name[variable]
            evidence_node_states[variable] = self._value_index(node, label)


        frontier = set(query) | set(evidence.keys())
        keep_vars = self._ancestors_of(network, frontier) | frontier


        # Build and restrict factors (only ancestors kept)
        factors = self._build_factors_for(network, keep_vars)
        for variable, idx in evidence_node_states.items():
            for factor in factors:
                factor.restrict(variable, idx)

        # Elimination order: topological over kept vars, drop evidence & query
        elim_order = [
            node.name for node in network.topological_order()
            if node.name in keep_vars
            and node.name not in evidence
            and node.name not in query
        ]


        # Variable elimination
        for elim in elim_order:
            bucket = [f for f in factors if elim in f.variables]
            if not bucket:
                continue
            new_factor = Factor(bucket[0].variables[:],
                                np.array(bucket[0].values, dtype=np.float32, copy=True))
            for f in bucket[1:]:
                new_factor = new_factor.multiply(f)
            new_factor.sum_out(elim)
            factors = [f for f in factors if f not in bucket] + [new_factor]


        # Multiply remaining factors
        if not factors:
            raise RuntimeError("No factors remain after elimination.")
        result = factors[0]
        for f in factors[1:]:
            result = result.multiply(f)

        # Sum out everything not in query
        for v in list(result.variables):
            if v not in query:
                result.sum_out(v)

        # Reorder + normalize
        result.reorder(query).normalize()
        return result

class GibbsSolver:
    """
    - network.by_name: dict[str, Node]
    - network.parents: dict[Node, set[Node]]
    - network.children: dict[Node, set[Node]]
    - Node: name(str), parents(tuple[str]), values(tuple[str]), probability_model(np.ndarray)
    """

    def __init__(self, iterations=2000, burn_in=500, thin=1, seed=None, verbose=False):
        assert iterations > 0 and 0 <= burn_in < iterations
        assert thin >= 1
        self.iterations = iterations
        self.burn_in = burn_in
        self.thin = thin
        self.rng = np.random.default_rng(seed)
        self.verbose = verbose

    def solve(self, network, report_var: str, evidence: dict[str, str]):

        if report_var in evidence:
            node = network.by_name[report_var]
            num_values = len(node.values)
            return Factor([report_var], [-1]*num_values)

        nodes_by_name = network.by_name                              # str -> Node
        report_node = nodes_by_name[report_var]

        state = {}
        for name, node in nodes_by_name.items():
            if name in evidence:
                state[name] = evidence[name]
            else:
                state[name] = self.rng.choice(node.values)

        non_evidence_nodes = [nodes_by_name[n] for n in nodes_by_name if n not in evidence]

        tally = Counter()
        kept = 0
        try:
            from tqdm import tqdm
            for it in tqdm(range(self.iterations), desc=f"Gibbs sampling: {report_var}"):
                for X_node in non_evidence_nodes:
                    X_name = X_node.name
                    x_vals = X_node.values
                    weights = np.empty(len(x_vals), dtype=float)

                    # Score each candidate value (Markov blanket go brrr)
                    for i, x in enumerate(x_vals):
                        old_val = state[X_name]
                        state[X_name] = x

                        # p(X=x | Pa(X))
                        w = self._cpd_prob(X_node, state, nodes_by_name)

                        for Y_node in network.children.get(X_node, ()):
                            w *= self._cpd_prob(Y_node, state, nodes_by_name)

                        weights[i] = w if (w > 0 and np.isfinite(w)) else 0.0
                        state[X_name] = old_val

                    s = weights.sum()
                    probs = (weights / s) if s > 0 else np.full(len(x_vals), 1.0 / len(x_vals))

                    # Samples new value for X
                    state[X_name] = self.rng.choice(x_vals, p=probs)

                if it >= self.burn_in and ((it - self.burn_in) % self.thin == 0):
                    tally[state[report_var]] += 1
                    kept += 1
        except Exception:
            for it in range(self.iterations):
                for X_node in non_evidence_nodes:
                    X_name = X_node.name
                    x_vals = X_node.values
                    weights = np.empty(len(x_vals), dtype=float)

                    # Score each candidate value (Markov blanket go brrr)
                    for i, x in enumerate(x_vals):
                        old_val = state[X_name]
                        state[X_name] = x

                        # p(X=x | Pa(X))
                        w = self._cpd_prob(X_node, state, nodes_by_name)

                        for Y_node in network.children.get(X_node, ()):
                            w *= self._cpd_prob(Y_node, state, nodes_by_name)

                        weights[i] = w if (w > 0 and np.isfinite(w)) else 0.0
                        state[X_name] = old_val

                    s = weights.sum()
                    probs = (weights / s) if s > 0 else np.full(len(x_vals), 1.0 / len(x_vals))

                    # Samples new value for X
                    state[X_name] = self.rng.choice(x_vals, p=probs)

                if it >= self.burn_in and ((it - self.burn_in) % self.thin == 0):
                    tally[state[report_var]] += 1
                    kept += 1

        if kept == 0:
            tally[state[report_var]] += 1
            kept = 1

        report_values = report_node.values
        counts = np.array([tally[v] for v in report_values], dtype=float)
        probs = counts / counts.sum()

        return Factor([report_var], np.array(probs, dtype=np.float32))

    def _cpd_prob(self, node, state, nodes_by_name):
        """
        Return p(node = state[node.name] | parents(node) = state[...])
        """
        pm = node.probability_model
        par_names = node.parents or ()

        # child outcome index
        x_label = state[node.name]
        try:
            x_idx = node.values.index(x_label)
        except ValueError:
            return 0.0

        # parent indices
        par_sizes, par_indices = [], []
        for p in par_names:
            domain = nodes_by_name[p].values
            pv = state[p]
            try:
                idx = domain.index(pv)
            except ValueError:
                return 0.0
            par_sizes.append(len(domain))
            par_indices.append(idx)

        # Multi-dimensional case
        if pm.ndim == 1 + len(par_names):
            try:
                return float(pm[(x_idx, *par_indices)])
            except Exception:
                return 0.0

        # 2-D tabular case
        if pm.ndim == 2:
            if len(par_names) == 0:
                if pm.shape[1] == 1:
                    return float(pm[x_idx, 0])
                if pm.shape[0] == len(node.values):
                    return float(pm[x_idx])
                return 0.0
            try:
                col = 0
                for i, b in zip(reversed(par_indices), reversed(par_sizes)):
                    col = col * b + i
                return float(pm[x_idx, col])
            except Exception:
                return 0.0

        # Unknown layout
        return 0.0

class OutputWriter:
    def __init__(self, output: List[List[Union[str, int]]]):
        os.makedirs("outputs", exist_ok=True)
        name = os.path.splitext(os.path.basename(NETWORK_NAME))[0]
        file_path = f"outputs/group{GROUP_ID}_{ALGORITHM}_{name}_{EVIDENCE_LEVEL}.csv"
        print(f"Writing to {file_path}")

        with open(file_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerows(output)

class Driver:
    def __init__(self):
        self.reader = InputReader()
        match(ALGORITHM.lower()):
            case "ve":
                self.solver = VESolver()
            case "gibbs":
                self.solver = GibbsSolver(iterations=10_000, burn_in=1000, thin=5, seed=GROUP_ID)
            case _:
                raise NotImplementedError

        marginals = [self.solver.solve(self.reader.network,report,self.reader.parsed_evidence) for report in self.reader.parsed_report]
        output = []
        for f in marginals:
            var = f.variables[0]
            output.append([var] + list(self.reader.network.by_name[var].values))
            if all([i == -1 for i in f.values]):
                output.append(['x']*len(f.values))
            else:
                output.append([f"{n:.2f}" for n in f.values])


        for line in output:
            print(", ".join(line))

        OutputWriter(output)

if __name__ == '__main__':
    driver = Driver()

Gibbs sampling: PropCost: 100%|██████████| 10000/10000 [00:08<00:00, 1234.82it/s]

[0.9216667  0.03222222 0.02611111 0.02      ]
[0.9811111  0.01       0.00555556 0.00333333]
[0.61777776 0.3061111  0.06944445 0.00666667]
MedCost, Thousand, TenThou, HundredThou, Million
0.92, 0.03, 0.03, 0.02
ILiCost, Thousand, TenThou, HundredThou, Million
0.98, 0.01, 0.01, 0.00
PropCost, Thousand, TenThou, HundredThou, Million
0.62, 0.31, 0.07, 0.01
Writing to outputs/group29_gibbs_insurance_None.csv
